In [8]:
import pandas as pd
import numpy as np

pd.set_option('display.max_columns', None)   # show every column
pd.set_option('display.width', 0)            # no line-wrapping


df_BTS2 = pd.read_csv("/Users/roskat/Desktop/CS427/data/BTS_1.csv") # Used for offline baseline training first
df_BTS1 = pd.read_csv("/Users/roskat/Desktop/CS427/data/BTS_2.csv") # This will be used for the incremental learning experiment

/var/folders/v3/9vhsqmm10zd5wzyzbk2jb9wr0000gn/T/ipykernel_96093/2972276224.py:8: DtypeWarning: Columns (12) have mixed types. Specify dtype option on import or set low_memory=False.
  df_BTS2 = pd.read_csv("/Users/roskat/Desktop/CS427/data/BTS_1.csv") # Used for offline baseline training first
/var/folders/v3/9vhsqmm10zd5wzyzbk2jb9wr0000gn/T/ipykernel_96093/2972276224.py:9: DtypeWarning: Columns (12) have mixed types. Specify dtype option on import or set low_memory=False.
  df_BTS1 = pd.read_csv("/Users/roskat/Desktop/CS427/data/BTS_2.csv") # This will be used for the incremental learning experiment


BTS2 Cleaning

Overviewing BTS2 data

In [9]:
df_BTS2.head()

,Flgs,Seq,Dur,RunTime,Mean,Sum,Min,Max,Proto,sTos,dTos,sDSb,dDSb,sTtl,dTtl,sHops,dHops,Cause,TotPkts,SrcPkts,DstPkts,TotBytes,SrcBytes,DstBytes,Offset,sMeanPktSz,dMeanPktSz,Load,SrcLoad,DstLoad,Loss,SrcLoss,DstLoss,pLoss,SrcGap,DstGap,Rate,SrcRate,DstRate,State,SrcWin,DstWin,sVid,dVid,SrcTCPBase,DstTCPBase,TcpRtt,SynAck,AckDat,Label,Attack Type,Attack Tool
0,e,1,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,icmp,0.0,NaN,cs0,NaN,58.0,NaN,6.0,NaN,Start,1,1,0,98,98,0,128,98.000000,0.000000,0.0000,0.0000,0.000000,0,0,0,0.0,NaN,NaN,0.000000,0.000000,0.000000,ECO,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,Benign,Benign,Benign
1,e,2,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,icmp,0.0,NaN,cs0,NaN,58.0,NaN,6.0,NaN,Start,1,1,0,98,98,0,232,98.000000,0.000000,0.0000,0.0000,0.000000,0,0,0,0.0,NaN,NaN,0.000000,0.000000,0.000000,ECO,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,Benign,Benign,Benign
2,e,3,4.998020,4.998020,4.998020,4.998020,4.998020,4.998020,udp,0.0,0.0,cs0,cs0,117.0,64.0,11.0,0.0,Start,214,196,18,249093,244212,4881,336,1245.979614,271.166656,396280.1250,388901.1875,7378.921875,0,0,0,0.0,NaN,NaN,42.616875,39.015450,3.401347,CON,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,Benign,Benign,Benign
3,e,4,4.998037,4.998037,4.998037,4.998037,4.998037,4.998037,udp,0.0,0.0,cs0,cs0,117.0,64.0,11.0,0.0,Start,184,163,21,221712,216245,5467,440,1326.656494,260.333344,352339.9375,344005.4688,8334.472656,0,0,0,0.0,NaN,NaN,36.614376,32.412727,4.001571,CON,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,Benign,Benign,Benign
4,e,5,4.999453,4.999453,4.999453,4.999453,4.999453,4.999453,udp,0.0,0.0,cs0,cs0,117.0,64.0,11.0,0.0,Start,223,204,19,280216,275723,4493,544,1351.583374,236.473679,445855.1563,439043.2188,6811.945312,0,0,0,0.0,NaN,NaN,44.404858,40.604443,3.600394,CON,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,Benign,Benign,Benign


In [13]:
df_BTS2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 728316 entries, 0 to 728315
Data columns (total 52 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   Flgs         728316 non-null  object 
 1   Seq          728316 non-null  int64  
 2   Dur          728316 non-null  float64
 3   RunTime      728316 non-null  float64
 4   Mean         728316 non-null  float64
 5   Sum          728316 non-null  float64
 6   Min          728316 non-null  float64
 7   Max          728316 non-null  float64
 8   Proto        728316 non-null  object 
 9   sTos         728116 non-null  float64
 10  dTos         146863 non-null  float64
 11  sDSb         728116 non-null  object 
 12  dDSb         146863 non-null  object 
 13  sTtl         728116 non-null  float64
 14  dTtl         146863 non-null  float64
 15  sHops        728116 non-null  float64
 16  dHops        146863 non-null  float64
 17  Cause        728316 non-null  object 
 18  TotPkts      728316 non-

In [21]:
df_BTS2.describe()

,Seq,Dur,RunTime,Mean,Sum,Min,Max,sTos,dTos,sTtl,dTtl,sHops,dHops,TotPkts,SrcPkts,DstPkts,TotBytes,SrcBytes,DstBytes,Offset,sMeanPktSz,dMeanPktSz,Load,SrcLoad,DstLoad,Loss,SrcLoss,DstLoss,pLoss,SrcGap,DstGap,Rate,SrcRate,DstRate,SrcWin,DstWin,sVid,dVid,SrcTCPBase,DstTCPBase,TcpRtt,SynAck,AckDat
count,728316.000000,728316.000000,728316.000000,728316.000000,728316.000000,728316.000000,728316.000000,728116.000000,146863.000000,728116.000000,146863.000000,728116.000000,146863.000000,728316.000000,728316.000000,728316.000000,7.283160e+05,7.283160e+05,7.283160e+05,7.283160e+05,728316.000000,728316.000000,7.283160e+05,7.283160e+05,7.283160e+05,728316.000000,728316.000000,728316.000000,728316.000000,145501.000000,145501.000000,7.283160e+05,7.283160e+05,7.283160e+05,1.252690e+05,9.079700e+04,72986.0,753.0,1.455010e+05,1.196160e+05,728316.000000,728316.000000,728316.000000
mean,51282.395176,1.426883,1.426883,1.426883,1.426883,1.426883,1.426883,0.753969,2.836971,82.136318,65.788306,2.232481,4.976747,5.051927,3.643325,1.408601,3.626844e+03,2.475103e+03,1.151741e+03,1.456031e+07,73.049689,56.555638,5.222136e+06,1.515005e+05,5.070636e+06,0.020654,0.012511,0.008150,0.344715,0.000007,0.766655,1.015731e+03,2.605478e+02,5.327791e+02,1.100917e+06,6.977458e+04,610.0,610.0,2.032239e+09,2.143869e+09,0.004013,0.000387,0.003626
std,42867.611959,1.750249,1.750249,1.750249,1.750249,1.750249,1.750249,11.620346,22.069261,56.768312,28.284148,3.392304,2.474563,25.399459,18.280360,13.283983,3.073910e+04,2.420562e+04,1.807014e+04,1.201207e+07,143.610527,207.539107,5.786276e+08,1.417473e+07,5.673006e+08,0.241640,0.167230,0.148771,3.681232,0.002622,57.615776,8.631973e+04,2.344623e+04,5.463027e+04,5.615949e+06,1.927221e+05,0.0,0.0,1.238678e+09,1.244851e+09,0.011560,0.004455,0.009933
min,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,36.000000,37.000000,0.000000,0.000000,1.000000,0.000000,0.000000,4.200000e+01,0.000000e+00,0.000000e+00,1.280000e+02,0.000000,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,610.0,610.0,2.310820e+05,4.385080e+05,0.000000,0.000000,0.000000
25%,10953.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,63.000000,59.000000,1.000000,5.000000,1.000000,1.000000,0.000000,4.200000e+01,4.200000e+01,0.000000e+00,3.683682e+06,42.000000,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,5.670400e+04,6.476800e+04,610.0,610.0,9.914339e+08,1.070841e+09,0.000000,0.000000,0.000000
50%,36820.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,63.000000,59.000000,1.000000,5.000000,2.000000,1.000000,0.000000,8.400000e+01,7.700000e+01,0.000000e+00,1.192173e+07,42.000000,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,6.259200e+04,6.489600e+04,610.0,610.0,1.927080e+09,2.152925e+09,0.000000,0.000000,0.000000
75%,90470.000000,2.581113,2.581113,2.581113,2.581113,2.581113,2.581113,0.000000,0.000000,63.000000,59.000000,1.000000,5.000000,2.000000,2.000000,0.000000,8.400000e+01,8.400000e+01,0.000000e+00,2.433654e+07,67.000000,0.000000,1.306236e+02,1.305918e+02,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,3.890480e-01,3.885980e-01,0.000000e+00,6.425600e+04,6.502400e+04,610.0,610.0,3.102191e+09,3.223245e+09,0.000000,0.000000,0.000000
max,137210.000000,19.920630,19.920630,19.920630,19.920630,19.920630,19.920630,224.000000,186.000000,255.000000,255.000000,28.000000,50.000000,3997.000000,986.000000,3151.000000,4.656256e+06,1.343477e+06,4.551675e+06,3.969216e+07,1392.000000,1478.000000,8.619200e+10,6.072000e+09,8.408000e+10,28.000000,15.000000,13.000000,62.500000,1.000000,7923.000000,1.300000e+07,4.000000e+06,8.000000e+06,3.355392e+07,1.677696e+07,610.0,610.0,4.29

In [ ]:
# Filling in all null values
df_BTS2 = df_BTS2.fillna(df_BTS2.median(numeric_only=True)) # Numeric values
df_BTS2[df_BTS2.select_dtypes('object').columns] = df_BTS2.select_dtypes('object').apply(lambda x: x.fillna(x.mode()[0])) # Object Values

In [29]:
df_BTS2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 728316 entries, 0 to 728315
Data columns (total 52 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   Flgs         728316 non-null  object 
 1   Seq          728316 non-null  int64  
 2   Dur          728316 non-null  float64
 3   RunTime      728316 non-null  float64
 4   Mean         728316 non-null  float64
 5   Sum          728316 non-null  float64
 6   Min          728316 non-null  float64
 7   Max          728316 non-null  float64
 8   Proto        728316 non-null  object 
 9   sTos         728316 non-null  float64
 10  dTos         728316 non-null  float64
 11  sDSb         728316 non-null  object 
 12  dDSb         728316 non-null  object 
 13  sTtl         728316 non-null  float64
 14  dTtl         728316 non-null  float64
 15  sHops        728316 non-null  float64
 16  dHops        728316 non-null  float64
 17  Cause        728316 non-null  object 
 18  TotPkts      728316 non-

BTS1 Cleaning

In [30]:
df_BTS1.head()

,Flgs,Seq,Dur,RunTime,Mean,Sum,Min,Max,Proto,sTos,dTos,sDSb,dDSb,sTtl,dTtl,sHops,dHops,Cause,TotPkts,SrcPkts,DstPkts,TotBytes,SrcBytes,DstBytes,Offset,sMeanPktSz,dMeanPktSz,Load,SrcLoad,DstLoad,Loss,SrcLoss,DstLoss,pLoss,SrcGap,DstGap,Rate,SrcRate,DstRate,State,SrcWin,DstWin,sVid,dVid,SrcTCPBase,DstTCPBase,TcpRtt,SynAck,AckDat,Label,Attack Type,Attack Tool
0,e,1,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,icmp,0.0,NaN,cs0,NaN,58.0,NaN,6.0,NaN,Start,1,1,0,98,98,0,128,98.000000,0.000000,0.0000,0.0000,0.000000,0,0,0,0.0,NaN,NaN,0.000000,0.000000,0.000000,ECO,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,Benign,Benign,Benign
1,e,2,4.994412,4.994412,4.994412,4.994412,4.994412,4.994412,udp,0.0,0.0,cs0,cs0,117.0,64.0,11.0,0.0,Start,182,157,25,215679,210561,5118,232,1341.152832,204.720001,342997.7500,335126.5313,7871.196777,0,0,0,0.0,NaN,NaN,36.240501,31.234909,4.805370,CON,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,Benign,Benign,Benign
2,e,3,4.997438,4.997438,4.997438,4.997438,4.997438,4.997438,udp,0.0,0.0,cs0,cs0,117.0,64.0,11.0,0.0,Start,233,212,21,286306,281547,4759,336,1328.051880,226.619049,455836.7813,448580.2500,7256.518066,0,0,0,0.0,NaN,NaN,46.423786,42.221634,4.002051,CON,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,Benign,Benign,Benign
3,e,4,4.999982,4.999982,4.999982,4.999982,4.999982,4.999982,udp,0.0,0.0,cs0,cs0,117.0,64.0,11.0,0.0,Start,225,213,12,291758,288185,3573,440,1352.981201,297.750000,464176.0938,458934.4688,5241.619141,0,0,0,0.0,NaN,NaN,44.800163,42.400154,2.200008,CON,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,Benign,Benign,Benign
4,e,5,4.961815,4.961815,4.961815,4.961815,4.961815,4.961815,udp,0.0,0.0,cs0,cs0,117.0,64.0,11.0,0.0,Start,161,145,16,197976,193641,4335,544,1335.455200,270.937500,316611.5938,310057.5313,6554.053223,0,0,0,0.0,NaN,NaN,32.246265,29.021639,3.023088,CON,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,Benign,Benign,Benign


In [32]:
df_BTS1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 487574 entries, 0 to 487573
Data columns (total 52 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   Flgs         487574 non-null  object 
 1   Seq          487574 non-null  int64  
 2   Dur          487574 non-null  float64
 3   RunTime      487574 non-null  float64
 4   Mean         487574 non-null  float64
 5   Sum          487574 non-null  float64
 6   Min          487574 non-null  float64
 7   Max          487574 non-null  float64
 8   Proto        487574 non-null  object 
 9   sTos         487560 non-null  float64
 10  dTos         125960 non-null  float64
 11  sDSb         487560 non-null  object 
 12  dDSb         125960 non-null  object 
 13  sTtl         487560 non-null  float64
 14  dTtl         125960 non-null  float64
 15  sHops        487560 non-null  float64
 16  dHops        125960 non-null  float64
 17  Cause        487574 non-null  object 
 18  TotPkts      487574 non-

In [33]:
df_BTS1 = df_BTS1.fillna(df_BTS1.median(numeric_only=True)) # Numeric values
df_BTS1[df_BTS1.select_dtypes('object').columns] = df_BTS1.select_dtypes('object').apply(lambda x: x.fillna(x.mode()[0])) # Object Values

In [34]:
df_BTS1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 487574 entries, 0 to 487573
Data columns (total 52 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   Flgs         487574 non-null  object 
 1   Seq          487574 non-null  int64  
 2   Dur          487574 non-null  float64
 3   RunTime      487574 non-null  float64
 4   Mean         487574 non-null  float64
 5   Sum          487574 non-null  float64
 6   Min          487574 non-null  float64
 7   Max          487574 non-null  float64
 8   Proto        487574 non-null  object 
 9   sTos         487574 non-null  float64
 10  dTos         487574 non-null  float64
 11  sDSb         487574 non-null  object 
 12  dDSb         487574 non-null  object 
 13  sTtl         487574 non-null  float64
 14  dTtl         487574 non-null  float64
 15  sHops        487574 non-null  float64
 16  dHops        487574 non-null  float64
 17  Cause        487574 non-null  object 
 18  TotPkts      487574 non-

In [36]:
df_BTS2.to_csv('dfBTS2_cleaned.csv', index = False)

In [37]:
df_BTS1.to_csv('dfBTS1_cleaned.csv', index = False)